# Summarize results for Marek's thesis

This notebook is used to generate latex tables for Marek's thesis.

## Main results

In [ ]:
import json
import os
import numpy as np
import pandas as pd

results_dir = "../results_thesis"
experiments = {
    # "eurlex_lightxml": "Eurlex",
    # "wiki10_100_lightxml": "Wiki10-31K",
    # "amazoncat_100_lightxml": "AmazonCat",
    # "wiki500_100_lightxml": "WikipediaLarge-500K",
    "rcv1x_100_plt": "RCV1x-2K",
    "eurlex_100_plt": "Eurlex-4K",
    #"lexglue_eurlex_100_plt": "LexGlue-Eurlex-5K",
    "wiki10_100_plt": "Wiki10-31K",
    "amazonCat_100_plt": "AmazonCat-13K",
    "deliciousLarge_100_plt": "DeliciousLarge-200K",
    "wikiLSHTC_100_plt": "WikiLSHTC-325K",
    "WikipediaLarge-500K_100_plt": "WikipediaLarge-500K",
    "amazon_100_lightxml": "Amazon-670K",
}

TOL = 1e-7
seeds = [13, 1988, 1993, 2023, 2024]
seeds = [13, 1988, 1993]
top_k = 3
val_split = 0.0
methods = {
    "optimal-instance-precision": "\\InfTopK",
    "optimal-instance-ps-precision": "\\InfPSK",
    "power-law-with-beta=0.5": "\\InfPowerK",
    "log": "\\InfLogK",
    "optimal-macro-balanced-accuracy": "\\InfMacBA",
    f"block-coord-macro-balanced-accuracy-tol={TOL}": "\\InfBCAMacBA",
    "frank-wolfe-macro-balanced-accuracy": "\\InfFWMacBA",
    f"block-coord-macro-precision-tol={TOL}": "\\InfBCAMacP",
    "frank-wolfe-macro-prec": "\\InfFWMacP",    
    "optimal-macro-recall": "\\InfMacR",
    f"block-coord-macro-recall-tol={TOL}": "\\InfBCAMacR",
    "frank-wolfe-macro-recall": "\\InfFWMacR",
    f"block-coord-macro-f1-tol={TOL}": "\\InfBCAMacF",
    "frank-wolfe-macro-f1": "\\InfFWMacF",
    f"block-coord-macro-jaccard-score-tol={TOL}": "\\InfBCAMacJ",
    "frank-wolfe-macro-jaccard-score": "\\InfFWMacJ",
    # f"block-coord-macro-gmean-tol={TOL}": "\\InfBCAMacGM",
    # "frank-wolfe-macro-gmean": "\\InfFWMacGM",
    # f"block-coord-macro-hmean-tol={TOL}": "\\InfBCAMacHM",
    # "frank-wolfe-macro-hmean": "\\InfFWMacHM",
    f"block-coord-coverage-tol={TOL}": "\\InfBCACov",
}

metrics = {
    "instance-precision": "P",
    "ps-precision": "PS",
    "instance-recall": "R",
    "macro-balanced-accuracy": "BA",
    "macro-precision": "P",
    "macro-recall": "R",
    "macro-f1": "F",
    "macro-jaccard-score": "JS",
    # "macro-gmean": "GM",
    # "macro-hmean": "HM",
    "coverage": "C"
}

multiplier = 100
format = "%.2f"

formats = ["\\textbf{{{:.2f}}}", "\\textit{{{:.2f}}}"]
    
for e, (experiment, experiment_label) in enumerate(experiments.items()):
    results = []
    for method, method_label in methods.items():
        method_results = {
            "method": method_label
        }
        for seed in seeds:
            filename = f"{results_dir}/{experiment}/{method}_k={top_k}_v={val_split}_s={seed}_results.json"
            if os.path.exists(filename):
                with open(filename, "r") as f:
                    result_file_data = json.load(f)
                for metric, metric_label in metrics.items():
                    metric = f"{metric}@{top_k}"
                    if metric in result_file_data:
                        method_results.setdefault(metric, []).append(result_file_data[metric] * multiplier)
            else:
                #print(f"File {filename} not found")
                pass

        for k, v in method_results.items():
            if isinstance(v, list) and isinstance(v[0], float):
                method_results[k] = np.mean(v)
        
        results.append(method_results)

    # Specializaed method worst results
    specialized_method_worst_results = {}
    for i, (method, method_label) in enumerate(methods.items()):
        for metric in metrics.keys():
            metric_at_k = f"{metric}@{top_k}"
            if metric in method and metric_at_k in results[i]:
                if isinstance(results[i][metric_at_k], float):
                    specialized_method_worst_results[metric_at_k] = min(specialized_method_worst_results.get(metric_at_k,100), results[i][metric_at_k])

    # Select best in column
    for metric in metrics.keys():
        metric_at_k = f"{metric}@{top_k}"
        column = [result.get(metric_at_k, 0) for result in results]
        argsort = np.argsort(column)[::-1]
        for result in results:
            if metric_at_k in result and isinstance(result[metric_at_k], float) and result[metric_at_k] < specialized_method_worst_results.get(metric_at_k,0):
                    result[metric_at_k] = f"{{\\color{{gray!75}} {result[metric_at_k]:.2f}}}"
        for format, idx in zip(formats, argsort):
            results[idx][metric_at_k] = format.format(column[idx])

    color = "green!25"
    # Color columns with target
    for i, (method, method_label) in enumerate(methods.items()):
        for metric in metrics.keys():
            metric_at_k = f"{metric}@{top_k}"
            if metric in method and metric_at_k in results[i]:
                if metric == "instance-recall":
                    color = "blue!25"
                if isinstance(results[i][metric_at_k], str):
                    results[i][metric_at_k] = f"\\cellcolor{{{color}}} {results[i][metric_at_k]}"
                else:
                    results[i][metric_at_k] = f"\\cellcolor{{{color}}} {results[i][metric_at_k]:.2f}"
    

    df = pd.DataFrame(results)

    if e == 0:
        custom_header = """
    \\begin{tabular}{l|rrr|rrrrrr}
    \\toprule
        Classifier & \\multicolumn{3}{c|}{Instance} & \\multicolumn{6}{c}{Macro} \\\\
        & \\multicolumn{1}{c}{P} & \\multicolumn{1}{c}{PS} & \\multicolumn{1}{c|}{R} 
        & \\multicolumn{1}{c}{BA} & \\multicolumn{1}{c}{P} & \\multicolumn{1}{c}{R} & 
        \\multicolumn{1}{c}{F$_1$} & \\multicolumn{1}{c}{JS} & \\multicolumn{1}{c}{Cov} \\\\"""
        print(custom_header)

    print(f"    \\midrule")
    print(f"    \\multicolumn{{10}}{{c}}{{{experiment_label}}} \\\\")

    # Print the results as a latex table
    latex_table = df.to_latex(index=False, float_format="{:0.2f}".format)
    print("\n".join(latex_table.split("\n")[3:-3]))

print("    \\bottomrule")
print("\\end{tabular}")


    \begin{tabular}{l|rrr|rrrrrr}
    \toprule
        Classifier & \multicolumn{3}{c|}{Instance} & \multicolumn{6}{c}{Macro} \\
        & \multicolumn{1}{c}{P} & \multicolumn{1}{c}{PS} & \multicolumn{1}{c|}{R} 
        & \multicolumn{1}{c}{BA} & \multicolumn{1}{c}{P} & \multicolumn{1}{c}{R} & 
        \multicolumn{1}{c}{F$_1$} & \multicolumn{1}{c}{JS} & \multicolumn{1}{c}{Cov} \\
    \midrule
    \multicolumn{10}{c}{Eurlex-4K} \\
\midrule
\InfTopK & \cellcolor{green!25} \textbf{68.32} & 154.42 & \textbf{40.46} & {\color{gray!75} 56.76} & {\color{gray!75} 21.99} & {\color{gray!75} 13.54} & {\color{gray!75} 15.75} & {\color{gray!75} 11.99} & {\color{gray!75} 28.30} \\
\InfPSK & {\color{gray!75} 67.79} & \cellcolor{green!25} \textit{173.58} & 40.09 & {\color{gray!75} 59.84} & {\color{gray!75} 27.23} & {\color{gray!75} 19.70} & 21.48 & 16.93 & {\color{gray!75} 36.19} \\
\InfPowerK & {\color{gray!75} 65.20} & \textbf{178.09} & 38.52 & {\color{gray!75} 60.98} & {\color{gray!75} 28.00} & {\